In [2]:
import sys
sys.path.append(r'C:\Users\Leuro\Desktop\DTU\Computer Vision\Exercice')
import utilis

import cv2 
import matplotlib.pyplot as plt
import numpy as np
import itertools as it
from scipy import linalg

![alt text](<Question/Screenshot 2024-05-14 090327.png>)

In [3]:
f = 1200
px = 400
py = 350 
alfa = 1
beta = 0
k3, k5 = 0.01, 0.04

K = np.array([[f, f*beta, px],
              [0, alfa*f, py],
              [0,0,1]])
print(K)

[[1200    0  400]
 [   0 1200  350]
 [   0    0    1]]


In [4]:
# Distortion coefficients (assuming radial distortion)
# Let's say we have three radial distortion coefficients and two tangential distortion coefficients
# dist_coeffs = np.array([k1, k2, p1, p2, k3])  # Example values for k1, k2, p1, p2, k3 need to be provided

# # Load an image
# img = cv2.imread('path_to_distorted_image.jpg')

# # Undistort the image
# undistorted_img = cv2.undistort(img, K, dist_coeffs)

#cv2.undistort() is used to correct the lens distortion in the image based on the provided camera matrix and distortion coefficients.
#the distortion coefficients handle the correction of the lens distortions that are not covered by the linear camera model.
# Radial distortion causes straight lines to appear curved.

![alt text](<Question/Screenshot 2024-05-14 090339.png>)

In [5]:
width = 800
height = 600
new_width = 400
new_height = 300

scale_factor_width = new_width/width
scale_factor_height = new_height/height
K = np.array([[1000*scale_factor_width, 0, 400*scale_factor_width],
              [0, 1000*scale_factor_height, 300*scale_factor_height],
              [0,0,1]])

K

array([[500.,   0., 200.],
       [  0., 500., 150.],
       [  0.,   0.,   1.]])

![alt text](<Question/Screenshot 2024-05-14 090359.png>)

In [6]:
f = 1720
px = 680
py = 610.0
alfa = 1
beta = 0

R = cv2.Rodrigues(np.array([-0.1, 0.1,-0.2]))[0]
t = np.array([[0.09], [0.05], [0.05]])
Q = np.array([[-0.03], [0.01], [0.59]])

K = np.array([[f, f*beta, px],
              [0, alfa*f, py],
              [0,0,1]])

q = utilis.projectpoints(K,R,t,Q)
q

array([[1023.50377104],
       [ 930.29756751]])

![alt text](<Question/Screenshot 2024-05-14 090413.png>)

In [7]:
K = np.array([[900, 0, 1070], [0, 900, 610.0], [0, 0, 1]], float) 
R1 = cv2.Rodrigues(np.array([-1.6, 0.3,-2.1]))[0] 
t1 = np.array([[0.0], [1.0], [3.0]], float) 
R2 = cv2.Rodrigues(np.array([-0.4,-1.3,-1.6]))[0] 
t2 = np.array([[0.0], [1.0], [6.0]], float) 
R3 = cv2.Rodrigues(np.array([2.5, 1.7,-0.4]))[0] 
t3 = np.array([[2.0], [-7.0], [25.0]], float)

q1 = np.array([[1046.0], [453.0]]) 
q2 = np.array([[1126.0], [671.0]]) 
q3 = np.array([[1165.0], [453.0]])

F12 = utilis.fundamental_matrix(R1,t1,R2,t2,K,K)
l  = utilis.epipolar_line(F12, utilis.PiInv(q1))
print(l.shape)
print(utilis.PiInv(q2).shape)
distance_numerator = np.abs(l.T@utilis.PiInv(q2))
distance = distance_numerator/(np.sqrt(l[0]**2 + l[1]**2))
distance

(3, 1)
(3, 1)


array([[13.27182907]])

![alt text](<Question/Screenshot 2024-05-14 090430.png>)

In [8]:
P1 = utilis.getProjection_Matrix(K,R1,t1)
P2 = utilis.getProjection_Matrix(K,R2,t2)
P3 = utilis.getProjection_Matrix(K,R3,t3)

q = np.hstack((q1,q2,q3))
P = [P1,P2,P3]
Q = utilis.triangulate(q,P)
Q

array([[3.10058867],
       [0.74321098],
       [0.46490561]])

![alt text](<Question/Screenshot 2024-05-14 090454.png>)

In [13]:
q = np.array([[2], [4], [3]])
l = np.array([[1], [2], [2]])

distance_numerator = np.abs(l.T@q)
print(distance_numerator)
distance = distance_numerator/(q[2]*(np.sqrt(l[0]**2 + l[1]**2)))
distance

[[16]]


array([[2.38513918]])

![alt text](<Question/Screenshot 2024-05-14 090507.png>)

In [10]:
harris = np.load("Question/harris.npy", allow_pickle=True).item()
harris.keys()

Ix_x = harris['g*(I_x^2)']
Iy_y = harris['g*(I_y^2)']
Ix_y = harris['g*(I_x I_y)']

k = 0.06
tao = 516

C = np.array([[Ix_x, Ix_y],
                  [Ix_y, Iy_y]])
a = C[0, 0]
b = C[1, 1]
c = C[0, 1]
r = a * b - c**2 - k * (a + b) ** 2

c = []
for i in range(1, r.shape[0] - 1):
    for j in range(1, r.shape[1] - 1):
        if (
            r[i, j] > r[i + 1, j]
            and r[i, j] >= r[i - 1, j]
            and r[i, j] > r[i, j + 1]
            and r[i, j] >= r[i, j - 1]
            and r[i, j] > tao
        ):  # Threshold
            c.append([i, j])
print(c)
# corners_index = np.where(r>tao )
# coordinates = [(x, y) for x, y in zip(corners_index[0], corners_index[1])]

# print("Coordinates:", coordinates)


[[2, 1]]


![alt text](<Question/Screenshot 2024-05-14 090516.png>)

In [11]:
ransac = np.load("Question/ransac.npy", allow_pickle=True).item()
ransac.keys()

points = ransac['points']
x1 = ransac['x1']
x2 = ransac['x2']

tao = 0.2
l = utilis.est_line(x1, x2)
inliers = utilis.consensus(l, points, tao)
inliers

34

![alt text](<Question/Screenshot 2024-05-14 090524.png>)

In [12]:
N = utilis.smallest_number_iterations(404,103, 0.95, 4)
N

707.5554825030302